# Shows how to Use the Data Alan Created amd Use Utility Functions

In [1]:
import pickle
from collections import namedtuple
import pandas as pd
import numpy as np

## Unpickle the Main Dataframe

In [2]:
df = pickle.load(open('dfu3.pkl', 'rb'))
df.head()

,site_id,vendor_code,service_type,cal_year,cal_mo,item_desc,units,cost,days_served,usage,fiscal_year,fiscal_mo,mmbtu
0,ANSBG1,VF314940,Natural Gas,2010,7,Natural gas (CCF),CCF,486.121875,2.5,212.187500,2011,1,21.643125
1,ANSBG1,VF314940,Natural Gas,2010,7,Other Charge,-,3.029688,2.5,NaN,2011,1,NaN
2,ANSBG1,VF314940,Natural Gas,2010,8,Natural gas (CCF),CCF,6127.112480,31.0,2674.425403,2011,2,272.791391
3,ANSBG1,VF314940,Natural Gas,2010,8,Other Charge,-,37.936442,31.0,NaN,2011,2,NaN
4,ANSBG1,VF314940,Natural Gas,2010,9,Natural gas (CCF),CCF,7842.075128,30.0,3422.990545,2011,3,349.145036


## Add a Degree Day column to a DataFrame

In [3]:
from importlib import reload
import sys

# bench_util module is in the parent directory, because it will be part of
# the production code.  Add the parent directory to the Path so Python can
# import from there
sys.path.insert(0, '../')   
import bench_util

In [33]:
# this is only needed to update any code changes I may have made
# since last importing the module.
reload(bench_util)     

# Make an object that has the various utility functions.
# The object needs access to the main utility bill data file and the spreadsheet
# containing other application data.
ut = bench_util.Util('data/ExampleDataExport.csv', '../data/Other_Building_Data.xlsx')

In [4]:
# this is only needed to update any code changes I may have made
# since last importing the module.
reload(bench_util)     

# Make an object that has the various utility functions.
# The object needs access to the main utility bill data file and the spreadsheet
# containing other application data.
ut = bench_util.Util('data/ExampleDataExport.csv', '../data/Other_Building_Data.xlsx')

# Add a Degree Day column to the df DataFrame. Just using this DataFrame for
# illustration here.  Normally you would add degree days in once you summed
# heating fuel across the entire building.
ut.add_degree_days_col(df)
df.head()

# This ut Util object can be reused for adding degree-days to other DataFrames.
# You only need to create it once in your script.

,site_id,vendor_code,service_type,cal_year,cal_mo,item_desc,units,cost,days_served,usage,fiscal_year,fiscal_mo,mmbtu,degree_days
0,ANSBG1,VF314940,Natural Gas,2010,7,Natural gas (CCF),CCF,486.121875,2.5,212.187500,2011,1,21.643125,134
1,ANSBG1,VF314940,Natural Gas,2010,7,Other Charge,-,3.029688,2.5,NaN,2011,1,NaN,134
2,ANSBG1,VF314940,Natural Gas,2010,8,Natural gas (CCF),CCF,6127.112480,31.0,2674.425403,2011,2,272.791391,169
3,ANSBG1,VF314940,Natural Gas,2010,8,Other Charge,-,37.936442,31.0,NaN,2011,2,NaN,169
4,ANSBG1,VF314940,Natural Gas,2010,9,Natural gas (CCF),CCF,7842.075128,30.0,3422.990545,2011,3,349.145036,568


## Get additional Information on a Building

In [5]:
# use the Util object from before to access additional Building Information.

# Value retunred below is a "named tuple"
rec = ut.bldg_info['ANSBG1']
rec

BldgInfo(site_name='Animal Control', address='2408 Davis Road', city='Fairbanks', primary_func='Animal Shelter', year_built=1993, sq_ft=14000, onsite_gen=nan, dd_site='PAFA')

In [6]:
# You can get individual fields through use of the dot operator
print(rec.address, ', ', rec.year_built)

2408 Davis Road ,  1993


## DataFrame for "Utility Cost Overview" Report Page

In [36]:
site = 'ANSBG1'
# From the main DataFrame, get only the rows for this site, and only get
# the needed columns for this analysis
df1 = df.query('site_id == @site')[['service_type', 'fiscal_year', 'fiscal_mo', 'cost']]
df1.head()

,service_type,fiscal_year,fiscal_mo,cost
0,Natural Gas,2011,1,486.121875
1,Natural Gas,2011,1,3.029688
2,Natural Gas,2011,2,6127.112480
3,Natural Gas,2011,2,37.936442
4,Natural Gas,2011,3,7842.075128


In [37]:
df2 = pd.pivot_table(
    df1, 
    values='cost', 
    index=['fiscal_year'], 
    columns=['service_type'],
    aggfunc=np.sum
)
df2

service_type,Electricity,Natural Gas,Sewer,Water
fiscal_year,,,,
2011,33894.895909,110858.416500,4716.576852,4379.291667
2012,41994.697841,94490.242371,5484.028148,5090.420833
2013,36961.451595,79970.329194,5169.324500,4783.854500
2014,35843.822155,48967.142581,4840.116833,4375.768167
2015,38017.419167,47128.677258,4593.016167,4156.942333
2016,29639.448485,45647.201628,6108.227500,5113.362500
2017,21849.234848,26682.420469,3326.440000,2875.330000


In [38]:
# Add a Total column that sums the other columns
df2['Total'] = df2.sum(axis=1)
df2

service_type,Electricity,Natural Gas,Sewer,Water,Total
fiscal_year,,,,,
2011,33894.895909,110858.416500,4716.576852,4379.291667,153849.180928
2012,41994.697841,94490.242371,5484.028148,5090.420833,147059.389193
2013,36961.451595,79970.329194,5169.324500,4783.854500,126884.959788
2014,35843.822155,48967.142581,4840.116833,4375.768167,94026.849736
2015,38017.419167,47128.677258,4593.016167,4156.942333,93896.054925
2016,29639.448485,45647.201628,6108.227500,5113.362500,86508.240113
2017,21849.234848,26682.420469,3326.440000,2875.330000,54733.425317


In [39]:
# Add a percent change column
df2['pct_change'] = df2.Total.pct_change()
df2

service_type,Electricity,Natural Gas,Sewer,Water,Total,pct_change
fiscal_year,,,,,,
2011,33894.895909,110858.416500,4716.576852,4379.291667,153849.180928,NaN
2012,41994.697841,94490.242371,5484.028148,5090.420833,147059.389193,-0.044133
2013,36961.451595,79970.329194,5169.324500,4783.854500,126884.959788,-0.137186
2014,35843.822155,48967.142581,4840.116833,4375.768167,94026.849736,-0.258960
2015,38017.419167,47128.677258,4593.016167,4156.942333,93896.054925,-0.001391
2016,29639.448485,45647.201628,6108.227500,5113.362500,86508.240113,-0.078681
2017,21849.234848,26682.420469,3326.440000,2875.330000,54733.425317,-0.367304


In [40]:
# Add in degree days
months_present = bench_util.months_present(df1)
deg_days = ut.degree_days_yearly(months_present, site)
df2['hdd'] = deg_days
df2

service_type,Electricity,Natural Gas,Sewer,Water,Total,pct_change,hdd
fiscal_year,,,,,,,
2011,33894.895909,110858.416500,4716.576852,4379.291667,153849.180928,NaN,13802
2012,41994.697841,94490.242371,5484.028148,5090.420833,147059.389193,-0.044133,12393
2013,36961.451595,79970.329194,5169.324500,4783.854500,126884.959788,-0.137186,12171
2014,35843.822155,48967.142581,4840.116833,4375.768167,94026.849736,-0.258960,13802
2015,38017.419167,47128.677258,4593.016167,4156.942333,93896.054925,-0.001391,12393
2016,29639.448485,45647.201628,6108.227500,5113.362500,86508.240113,-0.078681,12171
2017,21849.234848,26682.420469,3326.440000,2875.330000,54733.425317,-0.367304,12356


In [41]:
# Add in a column to show the numbers of months present for each year
# This will help to identify partial years.
mo_count = bench_util.month_count(months_present)
df2['month_count'] = mo_count
df2

service_type,Electricity,Natural Gas,Sewer,Water,Total,pct_change,hdd,month_count
fiscal_year,,,,,,,,
2011,33894.895909,110858.416500,4716.576852,4379.291667,153849.180928,NaN,13802,12
2012,41994.697841,94490.242371,5484.028148,5090.420833,147059.389193,-0.044133,12393,12
2013,36961.451595,79970.329194,5169.324500,4783.854500,126884.959788,-0.137186,12171,12
2014,35843.822155,48967.142581,4840.116833,4375.768167,94026.849736,-0.258960,13802,12
2015,38017.419167,47128.677258,4593.016167,4156.942333,93896.054925,-0.001391,12393,12
2016,29639.448485,45647.201628,6108.227500,5113.362500,86508.240113,-0.078681,12171,12
2017,21849.234848,26682.420469,3326.440000,2875.330000,54733.425317,-0.367304,12356,9


In [42]:
# Reverse the DataFrame
df2.sort_index(ascending=False, inplace=True)
df2

service_type,Electricity,Natural Gas,Sewer,Water,Total,pct_change,hdd,month_count
fiscal_year,,,,,,,,
2017,21849.234848,26682.420469,3326.440000,2875.330000,54733.425317,-0.367304,12356,9
2016,29639.448485,45647.201628,6108.227500,5113.362500,86508.240113,-0.078681,12171,12
2015,38017.419167,47128.677258,4593.016167,4156.942333,93896.054925,-0.001391,12393,12
2014,35843.822155,48967.142581,4840.116833,4375.768167,94026.849736,-0.258960,13802,12
2013,36961.451595,79970.329194,5169.324500,4783.854500,126884.959788,-0.137186,12171,12
2012,41994.697841,94490.242371,5484.028148,5090.420833,147059.389193,-0.044133,12393,12
2011,33894.895909,110858.416500,4716.576852,4379.291667,153849.180928,NaN,13802,12
